# Analysis on Stairway working hours

Based on Google Calendar data.

Exported to ICS file, then free converted to CSV using https://www.projectwizards.net/en/support/ics2csv-converter

In [ ]:
import pandas as pd
import numpy as np
import datetime

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

### Import and prep data

In [ ]:
!ls '../../data/google-calendar'

In [ ]:
column_names = ['title', 'start_time', 'end_time', 'notes', 'assigned_resources', 'additional_title']

df = pd.read_csv('../../data/google-calendar/stairway-to-travel-calendar.csv', 
                 sep="\t", names=column_names, header=0, parse_dates=[1, 2])

# create some usefull features
df['work_time'] = df['end_time'] - df['start_time']
df['work_hours'] = df['work_time'] / np.timedelta64(1, 'h')
df["week_date"] = (df['start_time'] - df['start_time'].dt.weekday * datetime.timedelta(days=1)).dt.date

In [ ]:
df.head()

### Simple stats

Period of administration

In [ ]:
df['start_time'].min(), df['start_time'].max()

In [ ]:
df['start_time'].max() - df['start_time'].min()

Total time invested in this period

In [ ]:
df['work_hours'].sum()

## Plot weekly working hours over time

Create an index to add missing weeks. Set working hours to zero for missing weeks.

In [ ]:
all_week_dates = pd.date_range(df["week_date"].min(), df["week_date"].max(), freq='W-MON')

In [ ]:
fig = plt.figure(figsize=(16,9))
ax = fig.gca()

hours_per_week = (
    df
    .groupby('week_date')
    .agg({
        'work_hours' : sum
    })
    .reindex(all_week_dates, fill_value=0)
    .assign(rolling_mean = lambda df: df.rolling(window=4).mean())
    .assign(overall_mean = lambda df: df['work_hours'].mean())
)

hours_per_week.plot(ax=ax);

Conclusie: Los van vakanties en met name onze bruiloft, gemiddeld >1 dag per week.

## Analyse time spent on

**TODO**: Find out how much time I spent on front-end, back-end or business work by extracting labels from text.

## Analyse when during the week I typically work

**TODO**: Make heatmap with hours of day on y-axis (00.00-24.00) and day of week on x-axis (Mon-Sun). Then light up the hours that I have worked.

Done.